In [ ]:
from IPython.display import clear_output
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install hazm
clear_output()

In [ ]:
import librosa
import json
import torch
import numpy as np
import IPython.display as ipd
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Tokenizer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = Wav2Vec2Processor.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-english")
model = Wav2Vec2ForCTC.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-english")
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
if torch.cuda.is_available():
    model.cuda()

clear_output()

In [ ]:
with open('dataset.json', 'r') as inFile:
    dataset = json.load(inFile)

utterances = dataset.keys()

for utterance in utterances:

    waveform, sample_rate = librosa.load(utterance, sr=16000)
    
    for chunk in dataset[utterance]:
        start_time = chunk["start_time"]
        end_time = chunk["end_time"]
        start_sample = int(start_time * sample_rate)
        end_sample = int(end_time * sample_rate)
        audio_segment = waveform[start_sample:end_sample]

        input_values = processor(audio_segment, sampling_rate=sample_rate, return_tensors="pt").input_values
        input_values = input_values.to(device)
        logits = model(input_values).logits
        predicted_ids = np.argmax(logits.cpu().detach().numpy(), axis=-1)
        transcription = processor.decode(predicted_ids[0])
        
        chunk["transcription"] = transcription

with open("transcribed_dataset.json", "w") as outFile:
    json.dump(dataset, outFile)

In [ ]:
extract_keywords('there is another general purpose robot in the works a start-up named figure has come out of stealth and is promising to build a humanoid similar to much-hyghped teslabot')

array([[ 6.76569045e-02,  6.34959415e-02,  4.87131290e-02,
         7.93049857e-02,  3.74480337e-02,  2.65279296e-03,
         3.93749774e-02, -7.09844474e-03,  5.93613945e-02,
         3.15370001e-02,  6.00980893e-02, -5.29051945e-02,
         4.06067558e-02, -2.59308461e-02,  2.98428219e-02,
         1.12692523e-03,  7.35148415e-02, -5.03819659e-02,
        -1.22386634e-01,  2.37028617e-02,  2.97265165e-02,
         4.24768627e-02,  2.56337933e-02,  1.99520262e-03,
        -5.69190867e-02, -2.71597859e-02, -3.29035521e-02,
         6.60248548e-02,  1.19007140e-01, -4.58791070e-02,
        -7.26214871e-02, -3.25839818e-02,  5.23413792e-02,
         4.50552963e-02,  8.25302955e-03,  3.67023908e-02,
        -1.39415218e-02,  6.53918907e-02, -2.64272466e-02,
         2.06386103e-04, -1.36643192e-02, -3.62809971e-02,
        -1.95043627e-02, -2.89738216e-02,  3.94270085e-02,
        -8.84090737e-02,  2.62427400e-03,  1.36714187e-02,
         4.83063422e-02, -3.11566107e-02, -1.17329143e-0

In [ ]:
from hazm import Normalizer, word_tokenize, sent_tokenize
from hazm import stopwords_list
from hazm import Stemmer
from hazm import word_tokenize
from yake import KeywordExtractor
import codecs
import collections
from multi_rake import Rake



def preprocess(text):
        # Normalize the text
    normalizer = Normalizer()
    normalized_text = normalizer.normalize(text)

    # Tokenize the text into words
    words = word_tokenize(normalized_text)

    # Remove stop words
    stopwords = stopwords_list()
    filtered_words = [word for word in words if word not in stopwords]

    #print(stemmed_words)
    result_text = " ".join(filtered_words)
    
# Text to extract keywords from
text = "پردازش زبان‌های طبیعی یکی از زیرشاخه‌های بااهمیت در حوزه‌ی گسترده‌ی علوم رایانه، هوش مصنوعی، که به تعامل بین کامپیوتر و زبان‌های (طبیعی) انسانی می‌پردازد؛ بنابراین پردازش زبان‌های طبیعی بر ارتباط انسان و رایانه، متمرکز است. پس چالش اصلی و عمده در این زمینه درک زبان طبیعی و ماشینی کردن فرایند درک و برداشت مفاهیم بیان‌شده با یک زبان طبیعیِ انسانی است. به تعریف دقیق‌تر، پردازش زبان‌های طبیعی عبارت است از استفاده از رایانه برای پردازش زبان گفتاری و زبان نوشتاری. بدین معنی که رایانه‌ها را قادر سازیم که گفتار یا نوشتار تولید شده در قالب و ساختار یک زبان طبیعی را تحلیل و درک نموده یا آن را تولید نمایند. در این صورت، با استفاده از آن می‌توان به ترجمهٔ زبان‌ها پرداخت، از صفحات وب و بانک‌های اطلاعاتیِ نوشتاری جهت پاسخ‌دادن به پرسش‌ها استفاده کرد، یا با دستگاه‌ها، مثلاً برای مشورت‌گرفتن به گفت‌وگو پرداخت. این‌ها تنها مثال‌هایی از کاربردهای متنوع پردازش زبان‌های طبیعی هستند. هدف اصلی در پردازش زبان طبیعی، ایجاد تئوری‌هایی محاسباتی از زبان، با استفاده از الگوریتم‌ها و ساختارهای داده‌ای موجود در علوم رایانه است. بدیهی است که در راستای تحقق این هدف، نیاز به دانشی وسیع از زبان است و علاوه بر محققان علوم رایانه، نیاز به دانش زبان‌شناسان نیز در این حوزه می‌باشد. با پردازش اطلاعات زبانی می‌توان آمار مورد نیاز برای کار با زبان طبیعی را استخراج کرد. کاربردهای پردازش زبان طبیعی به دو دسته کلی قابل تقسیم است: کاربردهای نوشتاری و کاربردهای گفتاری. از کاربردهای نوشتاری آن می‌توان به استخراج اطلاعاتی خاص از یک متن، ترجمه یک متن به زبانی دیگر یا یافتن مستنداتی خاص در یک پایگاه داده نوشتاری (مثلاً یافتن کتاب‌های مرتبط به هم در یک کتابخانه) اشاره کرد. نمونه‌هایی از کاربردهای گفتاری پردازش زبان عبارتند از: سیستم‌های پرسش و پاسخ انسان با رایانه، سرویس‌های اتوماتیک ارتباط با مشتری از طریق تلفن، سیستم‌های آموزش به فراگیران یا سیستم‌های کنترلی توسط صدا. در سالهای اخیر این حوزه تحقیقاتی توجه دانشمندان را به خود جلب کرده‌است و تحقیقات قابل ملاحظه‌ای در این زمینه صورت گرفته‌است."
def yake(text):
    
    preprocess(text)

    # Create a YAKE keyword extractor
    kw_extractor = KeywordExtractor()

    # Extract keywords from the text
    keywords = kw_extractor.extract_keywords(result_text)
    
    return keywords

# Print the keywords




def rake(text):
    
    preprocess(text)

    rake = Rake(
        min_chars=3,
        max_words=3,
        min_freq=1,
        language_code=None,  
        stopwords=None, 
        lang_detect_threshold=50,
        max_words_unknown_lang=2,
        generated_stopwords_percentile=80,
        generated_stopwords_max_len=3,
        generated_stopwords_min_freq=2,
    )
    keywords = rake.apply(result_text)
    return keywords

# Print the keywords

yake(text)


ModuleNotFoundError: ignored